# Serve Model

This notebook defines a prototype Flask web server with a REST API endpoint for scoring intances of data. The model-soring API endpoint uses a trained model that is downloaded from cloud storage (AWS S3) and loaded into memory.

# Imports and Configuration

In [1]:
from urllib.request import urlopen

import numpy as np
from flask import Flask, jsonify, make_response, request, Response
from joblib import load

## Load Model

In [2]:
model_url = (
    'http://bodywork-ml-ops-project.s3.eu-west-2.amazonaws.com'
    '/models/regressor-2021-01-12.joblib'
)
model = load(urlopen(model_url))
print(model)

LinearRegression()


## Define REST API Endpoint

Where a single instance of data will be POSTed to the `/score/v1` endpoint as JSON - e.g. `{"X": 50}`.

In [3]:
app = Flask(__name__)

@app.route('/score/v1', methods=['POST'])
def score_data_instance() -> Response:
    """Score incoming data instance using loaded model."""
    features = request.json['X']
    X = np.array(features, ndmin=2)
    prediction = model.predict(X)
    response_data = jsonify({'prediction': prediction[0], 'model_info': str(model)})
    return make_response(response_data)

## Start Test Server

Running the cell below will start the Flask test server and print the logs as results. To test the API endpoint defined above, open a terminal in another window and execute the following HTTP request using the curl tool,

```shell
curl http://0.0.0.0:5000/score/v1 \
    --request POST \
    --header "Content-Type: application/json" \
    --data '{"X": 50}'
```

The successful response from the model-scoring service should look like,

```json
{
    "prediction": 54.57560049377929,
    "model_info": "LinearRegression()"
}
```

Stop the service by interrupting the kernel (i.e. hit the 'stop button' above).

In [ ]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
